In [ ]:
import numpy as np
import pandas as pd
import os
import glob
import cv2

from sklearn.neighbors import KernelDensity

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

import utm
import rasterio

import random

import bat_functions as bf

In [ ]:
root_folder = ".../kasanka-bats/processed/deep-learning"
observations_root = os.path.join(root_folder, "observations")
all_observations = {}
day_folders = sorted(glob.glob(os.path.join(observations_root, '*')))
for day_folder in day_folders:
    obs_files = sorted(glob.glob(os.path.join(day_folder, '*.npy')))
    date = os.path.basename(day_folder)
    all_observations[date] = {}
    for obs_file in obs_files:
        camera = os.path.splitext(obs_file)[0].split('-')[-1]
        obs = np.load(obs_file, allow_pickle=True)
        # .item() to get dict from inside the array that was wrapped around
        # it when using np.save()
        all_observations[date][camera] = obs.item()



In [ ]:
# Remove observations to exclude (because camera ran out of batteries etc.)
exclude=True
# Manually exclude cameras that had issues
all_observations['17Nov']['MusoleParking']['exclude'] = True
all_observations['18Nov']['MusolePath']['exclude'] = True
all_observations['20Nov']['MusolePath']['exclude'] = True
if exclude:
    good_obs = {}
    for date, day_obs in all_observations.items():
        good_obs[date] = {}
        for camera, obs in day_obs.items():
            if 'exclude' in obs.keys():
                if obs['exclude']:
                    continue
            good_obs[date][camera] = obs
    all_observations = good_obs

In [ ]:
shift = 0 # loss on each side from not padding during detection (48)
FRAME_WIDTH = 2704 - (2 * shift)
WINGSPAN = .8 # meters, max extent while flying 

center_utm = {'middle': np.array([200450, 8606950]),
              'right': np.array([200800, 8606900])}

camera_locations = {'FibweParking2': [-12.5903393, 30.2525047],	
                    'FibweParking': [-12.5903393, 30.2525047],
                    'Chyniangale': [-12.5851284, 30.245529],	
                    'BBC': [-12.5863538, 30.2484985],
                    'Sunset': [-12.585784, 30.240003],
                    'NotChyniangale': [-12.5849206,	30.2436135],
                    'MusoleParking': [-12.58787, 30.2401],	
                    'MusolePath2': [-12.589544,	30.242488],	
                    'MusolePath': [-12.589544,	30.242488],
                    'Puku': [-12.584838, 30.24137],	
                    'FibwePublic': [-12.592537, 30.2515924],	
                    'MusoleTower': [-12.589434, 30.244736],
                    }

forest_border = [[-12.585957, 30.242762],
                 [-12.586763, 30.246229],
                 [-12.589182, 30.245566],
                 [-12.587557, 30.241598]
                ]

forest_utms = []
for f_latlon in forest_border:
    f_utm = utm.from_latlon(*f_latlon)
    forest_utms.append([f_utm[0], f_utm[1]])
forest_utms = np.array(forest_utms)

In [ ]:
map_file = '.../bats-data/maps/kasanka-utm.tiff'
forest_map_dataset = rasterio.open(map_file)
forest_map = [forest_map_dataset.read(band_ind) for band_ind in range(1, 4)]
forest_map = np.array(forest_map)
forest_map = forest_map.transpose(1, 2, 0)
width = np.abs(forest_map_dataset.bounds.left - forest_map_dataset.bounds.right).astype(int)
height = np.abs(forest_map_dataset.bounds.top - forest_map_dataset.bounds.bottom).astype(int)
area = np.zeros((height, width), dtype=np.uint8)

norm_forest = np.copy(forest_utms)
area_x_origin = forest_map_dataset.bounds.left
area_y_origin = forest_map_dataset.bounds.bottom
norm_forest[:, 0] = forest_utms[:, 0] - area_x_origin
norm_forest[:, 1] = forest_utms[:, 1] - area_y_origin
forest_mask = cv2.drawContours(area, 
                        [norm_forest.astype(np.int32)], 
                        -1, 255, -1)
plt.imshow(forest_mask)


In [ ]:
all_camera_utms = bf.latlong_dict_to_utm(camera_locations)

In [ ]:
save_plot_folder = '.../bats-data/plots/population-estimates'
os.makedirs(save_plot_folder, exist_ok=True)
plt.style.use('default')

In [ ]:
save_data_folder = '.../bats-data/counts'
os.makedirs(save_data_folder, exist_ok=True)


In [ ]:

def pick_observations(camera_dict, num_cameras):
    """ Randomly select num_cameras from camera_dict
     set num_cameras to -1 to return unchanged camera_dict"""
    if num_cameras == -1:
        return camera_dict
    keys = random.sample(camera_dict.keys(), num_cameras)
    new_dict = {}
    for k in keys:
        new_dict[k] = camera_dict[k]
    return new_dict

In [ ]:
replicates = 1000



vary_center = True
jitter = True
correct_wing = True

In [ ]:
# parameters for linear piecewise function for darkness error correction
parameters = [[1.57454778e+01, 9.37398964e-01, 7.18914388e-02, -1.27575036e-04],
#               [ 2.53107930e+01,  9.59547293e-01,  2.70747111e-02, -1.18602475e-03],
#               [1.03891791e+01, 8.78179573e-01, 1.86387502e-01, 1.77968688e-04]
             ]


# should_save=False
should_plot = False
save_counts = True
# save_folder = os.path.join(plot_folder, 'bat-count-error')
# os.makedirs(save_folder, exist_ok=True)

center = "middle"

fontsize = 15
num_cols = 5
max_bats = 0


dates = ['16Nov', '17Nov', '18Nov', '19Nov', '20Nov']
# dates = ['16Nov']

for num_cameras in [-1, 1, 2, 3, 4, 5, 6, 7, 8, 9]:
    print(f"num cameras {num_cameras}")
    trial_dicts = []
    for date in dates:
        for _ in range(replicates):
            if vary_center:

                trial_dict = {'date': date,
                              'observations': pick_observations(
                                  all_observations[date], num_cameras),
                              'center': bf.get_random_utm_in_mask(forest_mask, 
                                                               forest_map_dataset),
                              'dark_parameters': parameters[0],
                              'jitter': jitter,
                              'correct_wing': correct_wing,
                              'plot_color': 'b',
                              'accumulation': None,
                              'camera_fractions': None
                             }
            else:

                trial_dict = {'date': date,
                          'observations': all_observations[date],
                          'center': center_utm['middle'],
                          'dark_parameters': parameters[0],
                          'jitter': jitter,
                          'correct_wing': correct_wing,
                          'plot_color': 'b',
                          'accumulation': None,
                          'camera_fractions': None
                             }

            trial_dicts.append(trial_dict)

    wing_validation_file = '.../bats-data/wing-validation/combined_wing_validation_info.csv'
    wing_correction_info = bf.get_wing_correction_distributions(wing_validation_file,
                                                             num_darkness_bins=4,
                                                             kde_bw_scale=.25,
                                                             should_plot=False
                                                            )
    wing_correction_kdes, darkness_bins = wing_correction_info

    day_plots = {}
    day_totals = {}
    for date in dates:
        if should_plot:
            day_plots[date] = plt.subplots(2, num_cols, figsize=(20,20))
        day_totals[date] = []

    for trial_num, trial_dict in enumerate(trial_dicts):
        if trial_num % 500 == 0:
            print(f'trial num {trial_num} starting...')

        observations = trial_dict['observations']
        date = trial_dict['date']
        camera_utms = bf.get_camera_locations(observations, 
                                              all_camera_utms, 
                                              exclude=True)
        camera_distances = bf.get_camera_distances(camera_utms, 
                                                   trial_dict['center'])
        camera_fractions = bf.get_camera_fractions(camera_utms, 
                                                   trial_dict['center'],
                                                   trial_dict['jitter'])
        trial_dict['camera_fractions'] = camera_fractions

        dark_parameters = trial_dict['dark_parameters']
        if should_plot:
            fig, axs = day_plots[date]

        day_total = 0
        trial_summary = {}
        for cam_ind, (cam_name, obs) in enumerate(observations.items()):

            assert len(obs['darkness']) == len(obs['mean_wing'])
    #         darkness_means = np.load(
    #             os.path.join(root_folder, date, cam_name, 'blue-means.npy')
    #         )

    #         trial_dict['cameras'].append(cam_name)
            obs['fraction_total'] = camera_fractions[obs['camera']]
            if trial_dict['correct_wing']:
                correction_scale, kde_inds = bf.get_kde_samples(obs, wing_correction_kdes, darkness_bins)
            else:
                correction_scale = 0

            biased_wing = bf.correct_wingspan(obs['mean_wing'], correction_scale)
            biased_wing = np.maximum(biased_wing, 2) # No wingspans smaller than 2 pixels
            obs['multiplier'] = bf.combined_bat_multiplier(FRAME_WIDTH, 
                                                           WINGSPAN, 
                                                           biased_wing, 
                                                           camera_distances[obs['camera']]
                                                          )
            bat_accumulation = bf.get_bat_accumulation(obs['frames'], obs, dark_parameters)
            trial_summary[cam_name] = bat_accumulation[-1]

            if should_plot:
                axs[cam_ind//num_cols, cam_ind%num_cols].plot(
                    bat_accumulation, c=trial_dict['plot_color'], alpha=.1)
                axs[cam_ind//num_cols, cam_ind%num_cols].set_title(
                    cam_name, fontsize=fontsize*1.5)

            day_total += bat_accumulation[-1]

            if bat_accumulation[-1] > max_bats:
                max_bats = bat_accumulation[-1]
        day_totals[date].append(day_total)
        trial_summary['total'] = day_total
        trial_dict['accumulation'] = trial_summary
        
    if num_cameras == -1:
        num_cameras = 'all'
    count_file = f"num_cameras-{num_cameras}-jitter-{jitter}-vary_center-{vary_center}-correct_wing-{correct_wing}-replicates-{replicates}.npy"
    np.save(os.path.join(save_data_folder, count_file), day_totals)
max_bats += 10 
if should_plot:
    for date, plot in day_plots.items():  
        fig, axs = plot
        for ax_ind, ax in enumerate(axs.reshape(-1)):
            ax.set_ylim(top=max_bats)
            ax.set_xlabel('frame number', fontsize=fontsize)
            ax.tick_params(axis='y', labelsize=fontsize)
            ax.yaxis.set_major_formatter(
                mpl.ticker.StrMethodFormatter('{x:,.0f}'))
            if ax_ind % num_cols != 0:
                ax.tick_params(labelleft=False)  
        for r in range(len(axs)):
            axs[r, 0].set_ylabel('number of bats seen', fontsize=fontsize)
        fig.suptitle(f'{date} average total bats: {np.mean(day_totals[date]):,.0f}', size=fontsize*2)
        
        title = f"camera count distribution {date} jitter {jitter} correct_wing {correct_wing} vary_center {vary_center} replicates {replicates}"
        bf.save_fig(save_plot_folder, title, fig)


        plt.figure()
        plt.hist(day_totals[date], bins=15)
        plt.xlabel('Estimated number of bats')
        plt.ylabel('Number of simulations')
        plt.title(date)
        
        title = f"day count distribution {date} jitter {jitter} correct_wing {correct_wing} vary_center {vary_center} replicates {replicates}"
        bf.save_fig(save_plot_folder, title)
    


In [ ]:
num_cameras = 'all'
day_total_file = f"num_cameras-{num_cameras}-jitter-{jitter}-vary_center-{vary_center}-correct_wing-{correct_wing}-replicates-{replicates}.npy"
day_totals = np.load(os.path.join(save_data_folder, day_total_file), 
                     allow_pickle=True).item()

In [ ]:
random.choices(totals, k=10)

In [ ]:
def calc_mean(day_totals, replicates=1000):
    combined_estimates = []
    for _ in range(replicates):
        day_estimates = []
        for day, totals in day_totals.items():
            day_estimates.append(np.mean(random.choices(totals, k=1)))
        combined_estimates.append(np.mean(day_estimates))
    
    return np.mean(combined_estimates)

In [ ]:
sns.violinplot(means)

In [ ]:
mean_diffs = []

mean = 857932.4450041048

num_samples = 1000

for _ in range(num_samples):
    mean_diffs.append(calc_mean(day_totals) - mean)
sns.violinplot(mean_diffs)

In [ ]:
mean

In [ ]:
sorted_diffs = sorted(mean_diffs)
percentile = int(num_samples * .025)
print(sorted_diffs[percentile] + mean, sorted_diffs[-percentile] + mean)

In [ ]:
replicates = 1000
combined_estimates = []
for _ in range(replicates):
    day_estimates = []
    for day, totals in day_totals.items():
        day_estimates.append(np.mean(random.choices(totals, k=1)))
    combined_estimates.append(np.mean(day_estimates))
sns.violinplot(x=np.ones(replicates), y=combined_estimates)
print(np.mean(combined_estimates))

In [ ]:
num_cameras = 'all'
day_total_file = f"num_cameras-{num_cameras}-jitter-{jitter}-vary_center-{vary_center}-correct_wing-{correct_wing}-replicates-{replicates}.npy"
day_totals = np.load(os.path.join(save_data_folder, day_total_file), 
                     allow_pickle=True).item()

total_min = 10000000
total_max = 0
for day, totals in day_totals.items():
    print(f"{day}, mean: {np.mean(totals)}, min: {np.min(totals)}, max: {np.max(totals)}")
    if np.min(totals) < total_min:
        total_min = np.min(totals)
        
    if np.max(totals) > total_max:
        total_max = np.max(totals)
        
print(f"min {total_min},  max {total_max}")

In [ ]:
save = True

num_cameras = 'all'
day_total_file = f"num_cameras-{num_cameras}-jitter-{jitter}-vary_center-{vary_center}-correct_wing-{correct_wing}-replicates-{replicates}.npy"
day_totals = np.load(os.path.join(save_data_folder, day_total_file), 
                     allow_pickle=True).item()



all_totals = []
all_dates = []
for day, totals in day_totals.items():
    all_dates.extend([day for _ in totals])
    all_totals.extend(totals)
        
replicates_mean = 1000
combined_estimates = []
for _ in range(replicates_mean):
    day_estimates = []
    for day, totals in day_totals.items():
        day_estimates.append(np.mean(random.choices(totals, k=1)))
    combined_estimates.append(np.mean(day_estimates))
print(f"mean average {np.mean(combined_estimates)}")   
print(f"min average {np.min(combined_estimates)}") 
print(f"max average {np.max(combined_estimates)}") 
all_dates.extend(['Average' for _ in totals])
all_totals.extend(combined_estimates)


plt.figure()

day_color = "#858585"
total_color = "#FFFFFF"

colors = [day_color for _ in range(5)]
colors.append(total_color)
# Set your custom color palette
sns.set_palette(sns.color_palette(colors))

sns.violinplot(x=all_dates, y=all_totals)
plt.ylabel('Total bat population estimate')
plt.xlabel('Observation date')
plt.axvline(4.5, c='k', linestyle=':')
plt.ylim(0, 1400000)
plt.ticklabel_format(style='plain', axis='y')
plt.gca().get_yaxis().set_major_formatter(
    mpl.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
#     plt.title(f"num cameras: {num_cameras}, replicates {replicates}")
title = f"total count distribution violin {date} jitter {jitter} correct_wing {correct_wing} vary_center {vary_center} replicates {replicates}"
if save:
    bf.save_fig(save_plot_folder, title)

In [ ]:
np.mean(combined_estimates)

In [ ]:
save = False


min_val = 1000000000
min_day = None
max_val = 0
max_day = None

day = '16Nov'
for day in ['16Nov', '17Nov', '18Nov', '19Nov', '20Nov']:
    all_totals = []
    all_num_cameras = []
    for num_cameras in [1, 2, 3, 4, 5, 6, 7, 8, 9, 'all']:
        day_total_file = f"num_cameras-{num_cameras}-jitter-{jitter}-vary_center-{vary_center}-correct_wing-{correct_wing}-replicates-{replicates}.npy"
        day_totals = np.load(os.path.join(save_data_folder, day_total_file), 
                             allow_pickle=True).item()

        totals = day_totals[day]
        x_label = num_cameras
        if num_cameras == 'all':
            if day == '16Nov' or day == '19Nov':
                x_label = 10
            else:
                x_label = 9
        all_num_cameras.extend([x_label for _ in totals])
        all_totals.extend(totals)
        
        local_max = np.max(totals)
        local_min = np.min(totals)
        
        if local_max > max_val:
            max_val = local_max
            max_day = day
        if local_min < min_val:
            min_val = local_min
            min_day = day
    
    plt.figure(figsize=(12,4))

    sns.violinplot(x=all_num_cameras, y=all_totals, cut=0, color="gray")
    plt.ylabel('Total bat population estimate')
    plt.xlabel('Number of cameras used')
    plt.title(f"{day}")
    title = f"total count distribution violin varying camera num date {day} replicates {replicates}"
    if save:
        bf.save_fig(save_plot_folder, title)
        
print(f"min {min_val}, {min_day}, max: {max_val}, {max_day}")

In [ ]:
save_plot_folder

In [ ]:
camera_names = {'16Nov':['NotChyniangale', 'Chyniangale',
                         'BBC', 'FibweParking', 'FibwePublic',
                         'MusoleTower', 'MusolePath', 'MusoleParking',
                         'Sunset', 'Puku'],
                '17Nov': ['NotChyniangale', 'Chyniangale',
                         'BBC', 'FibweParking2', 'FibwePublic',
                         'MusoleTower', 'MusolePath2', 'MusoleParking',
                         'Sunset', 'Puku'],
                '18Nov': ['NotChyniangale', 'Chyniangale',
                         'BBC', 'FibweParking', 'FibwePublic',
                         'MusoleTower', 'MusoleParking',
                         'Sunset', 'Puku'],
                '19Nov': ['NotChyniangale', 'Chyniangale',
                         'BBC', 'FibweParking', 'FibwePublic',
                         'MusoleTower', 'MusolePath', 'MusoleParking',
                         'Sunset', 'Puku'],
                '20Nov': ['NotChyniangale', 'Chyniangale',
                         'BBC', 'FibweParking', 'FibwePublic',
                         'MusoleTower', 'MusoleParking',
                         'Sunset', 'Puku'],
               }

In [ ]:
camera_counts = {}

for date in dates:
    camera_counts[date] = {}

for trial in trial_dicts:
    date = trial['date']
    for camera, count in trial['accumulation'].items():
        if camera == 'total':
            continue
        bat_per_degree = count / (trial['camera_fractions'][camera] * 360)
        if camera in camera_counts[date].keys():
            camera_counts[date][camera].append(bat_per_degree)
        else:
            camera_counts[date][camera] = [bat_per_degree]

for date, day_camera_counts in camera_counts.items():
    
    fig, (ax1) = plt.subplots(1, 1)
    all_totals = []
    all_cameras = []
    for camera in camera_names[date]:
        total = day_camera_counts[camera]
        all_cameras.extend([camera for _ in total])
        all_totals.extend(total)
    sns.violinplot(x=all_cameras, y=all_totals)
    ax1.set_xticklabels(camera_names[date], rotation = 90)
    ax1.set_ylabel('Bats per degree')
    ax1.set_xlabel('Camera', )
    title = f"total count distribution violin {date} jitter {jitter} correct_wing {correct_wing} vary_center {vary_center} replicates {replicates}"
    bf.save_fig(save_plot_folder, title)

In [ ]:
for pos in all_camera_utms.values():
    plt.scatter(pos[0], pos[1])
# for trial_dict in trial_dicts:
for _ in range(500):
    utm = get_random_utm_in_mask(forest_mask, 
                           forest_map_dataset)
    plt.scatter(utm[0], 
                utm[1],
               s=1)
plt.scatter(center_utm['middle'][0], center_utm['middle'][1])

In [ ]:
correction_scale = kde.sample(obs['mean_wing'].shape)[:,0]
print(correction_scale.shape)
# else:
#     correction_scale = 0
biased_wing = correct_wingspan(obs['mean_wing'], correction_scale)
print(biased_wing.shape)
biased_wing = np.maximum(biased_wing, 2)
print(biased_wing.shape)

In [ ]:
plt.scatter(obs['mean_wing'], biased_wing, alpha=.1)
plt.scatter(np.arange(100), np.arange(100))

In [ ]:
# parameters for linear piecewise function for darkness error correction
parameters = [1.57454778e+01, 9.37398964e-01, 7.18914388e-02, -1.27575036e-04]
parameters_alt = [ 2.53107930e+01,  9.59547293e-01,  2.70747111e-02, -1.18602475e-03]
parameters_alt2 = [1.03891791e+01, 8.78179573e-01, 1.86387502e-01, 1.77968688e-04]
should_save=False
show_alt_params = True
save_folder = os.path.join(plot_folder, 'bat-accumulation')

center = "middle"

fontsize = 15

num_cols = 5

max_bats = 0
# so all plots have same scale
for date, day_obs in observations.items():
    camera_utms = bf.get_camera_locations(day_obs, 
                                       all_camera_utms, 
                                       exclude=True)
    camera_distances = bf.get_camera_distances(camera_utms, 
                                               center_utm[center])
    camera_fractions = bf.get_camera_fractions(camera_utms, center_utm[center])
    for obs in day_obs.values():
        if exclude and ('exclude' in obs.keys()):
            if obs['exclude']:
                continue
        obs['multiplier'] = bf.combined_bat_multiplier(FRAME_WIDTH, 
                                                    WINGSPAN, 
                                                    obs['mean_wing'], 
                                                    camera_distances[obs['camera']]
                                                   )
        obs['fraction_total'] = camera_fractions[obs['camera']]
    bf.get_day_total(day_obs, center_utm['middle'], 
                     all_camera_utms, FRAME_WIDTH, WINGSPAN, 
                     exclude=exclude, correct_darkness=True,
                     parameters=parameters
                    )                                         
    for cam_ind, (cam_name, obs) in enumerate(day_obs.items()):
        if exclude and ('exclude' in obs.keys()):
            if obs['exclude']:
                continue
        bat_accumulation = bf.get_bat_accumulation(obs['frames'], obs, parameters_alt)
        if bat_accumulation[-1] > max_bats:
            max_bats = bat_accumulation[-1]
max_bats += 1000  




for date, day_obs in observations.items():
    fig, axs = plt.subplots(2, num_cols, figsize=(20,20))
    
    total = 0
    total0 = 0
    total1 = 0
    total2 = 0
    for cam_ind, (cam_name, obs) in enumerate(day_obs.items()):
        darkness_means = np.load(
            os.path.join(root_folder, date, cam_name, 'blue-means.npy')
        )
        c0 = 'b'
        bat_accumulation = bf.get_bat_accumulation(obs['frames'], obs, parameters)
        axs[cam_ind//num_cols, cam_ind%num_cols].plot(bat_accumulation, label='corrected', c=c0)
        total0 += bat_accumulation[-1]
        bat_accumulation = bf.get_bat_accumulation(obs['frames'], obs, parameters, w_darkness=False)
        axs[cam_ind//num_cols, cam_ind%num_cols].plot(bat_accumulation, label='raw', c='k')
        total += bat_accumulation[-1]
        beginning_error_frame = np.argmax(darkness_means < parameters[0])
        axs[cam_ind//num_cols, cam_ind%num_cols].axvline(beginning_error_frame, c=c0)
        if show_alt_params:
            c1 = 'r'
            bat_accumulation = bf.get_bat_accumulation(obs['frames'], obs, parameters_alt)
            axs[cam_ind//num_cols, cam_ind%num_cols].plot(bat_accumulation, label='alt params', c=c1)
            beginning_error_frame = np.argmax(darkness_means < parameters_alt[0])
            axs[cam_ind//num_cols, cam_ind%num_cols].axvline(beginning_error_frame, c=c1)
            total1 += bat_accumulation[-1]
            
            c2 = 'g'
            bat_accumulation = bf.get_bat_accumulation(obs['frames'], obs, parameters_alt2)
            axs[cam_ind//num_cols, cam_ind%num_cols].plot(bat_accumulation, label='alt params', c=c2)
            beginning_error_frame = np.argmax(darkness_means < parameters_alt2[0])
            axs[cam_ind//num_cols, cam_ind%num_cols].axvline(beginning_error_frame, c=c2)
            total2 += bat_accumulation[-1]
        axs[cam_ind//num_cols, cam_ind%num_cols].set_title(cam_name, fontsize=fontsize*1.5)
    for ax_ind, ax in enumerate(axs.reshape(-1)):
        ax.set_ylim(top=max_bats)
        ax.set_xlabel('frame number', fontsize=fontsize)
        ax.tick_params(axis='y', labelsize=fontsize)
        ax.yaxis.set_major_formatter(
            mpl.ticker.StrMethodFormatter('{x:,.0f}'))
        if ax_ind % num_cols != 0:
            ax.tick_params(labelleft=False)  
    for r in range(len(axs)):
        axs[r, 0].set_ylabel('number of bats seen', fontsize=fontsize)
    fig.suptitle(f'{date} total bats: \n raw: {total:,.0f}, blue: {total0:,.0f}, red: {total1:,.0f}, green: {total2:,.0f}', size=fontsize*2)
    if show_alt_params:
        plot_name = 'bat-accumulation-scaled-break-{}-comparison-{}.png'.format(parameters[0], date)
    else:
        plot_name = 'bat-accumulation-scaled-break-{}-{}.png'.format(parameters[0], date)
    plot_file = os.path.join(save_folder, plot_name)
    if should_save:
        fig.savefig(plot_file, bbox_inches='tight')

In [ ]:
obs.keys()